In [2]:
# Quantization code from Song Han's TinyML class
import torch

import matplotlib.pyplot as plt

import numpy as np

from PIL import Image

def get_quantized_range(bitwidth):
    quantized_max = (1 << (bitwidth - 1)) - 1
    quantized_min = -(1 << (bitwidth - 1))
    return quantized_min, quantized_max

def get_quantization_scale_and_zero_point(fp_tensor, bitwidth):
    """
    get quantization scale for single tensor
    :param fp_tensor: [torch.(cuda.)Tensor] floating tensor to be quantized
    :param bitwidth: [int] quantization bit width
    :return:
        [float] scale
        [int] zero_point
    """
    quantized_min, quantized_max = get_quantized_range(bitwidth)
    fp_max = fp_tensor.max()
    fp_min = fp_tensor.min()

    # scale
    scale = (fp_max - fp_min) / (quantized_max - quantized_min)
    # zero_point
    zero_point = ((quantized_min - fp_min / scale))

    # clip the zero_point to fall in [quantized_min, quantized_max]
    if zero_point < quantized_min:
        zero_point = quantized_min
    elif zero_point > quantized_max:
        zero_point = quantized_max
    else: # convert from float to int using round()
        zero_point = round(zero_point)
    return scale, int(zero_point)

def linear_quantize(fp_tensor, bitwidth, scale, zero_point, dtype=np.int8) -> np.array:
    """
    linear quantization for single fp_tensor
      from
        r = fp_tensor = (quantized_tensor - zero_point) * scale
      we have,
        q = quantized_tensor = int(round(fp_tensor / scale)) + zero_point
    :param tensor: [np.array] floating tensor to be quantized
    :param bitwidth: [int] quantization bit width
    :param scale: [float] scaling factor
    :param zero_point: [int] the desired centroid of tensor values
    :return:
        [np.array] quantized tensor whose values are integers
    """
    # assert(fp_tensor is np.array)
    assert(isinstance(scale, float))
    assert(isinstance(zero_point, int))

    # scale the fp_tensor
    scaled_tensor = fp_tensor/scale
    # round the floating value to integer value
    rounded_tensor = (np.round(scaled_tensor)) #.to(torch.int8)

    # print(rounded_tensor.dtype)

    # shift the rounded_tensor to make zero_point 0
    shifted_tensor = rounded_tensor + zero_point

    # clamp the shifted_tensor to lie in bitwidth-bit range
    quantized_min, quantized_max = get_quantized_range(bitwidth)
    quantized_tensor = shifted_tensor.clip(quantized_min, quantized_max)
    quantized_tensor = quantized_tensor.astype(np.int8)
    return quantized_tensor

def linear_quantize_feature(fp_tensor, bitwidth):
    """
    linear quantization for feature tensor
    :param fp_tensor: [torch.(cuda.)Tensor] floating feature to be quantized
    :param bitwidth: [int] quantization bit width
    :return:
        [torch.(cuda.)Tensor] quantized tensor
        [float] scale tensor
        [int] zero point
    """
    scale, zero_point = get_quantization_scale_and_zero_point(fp_tensor, bitwidth)
    quantized_tensor = linear_quantize(fp_tensor, bitwidth, scale, zero_point)
    return quantized_tensor, scale, zero_point

checkpoint = torch.load("cpu/NN/mnist_100_10.pt")
# checkpoint = torch.load("./NN/MNIST_12_layers.pt")

weights_biases = {}

for name, param in checkpoint.items():
    weights_biases[name] = param.cpu().numpy()  # Convert to numpy array and store

int8_quant = {}

for key in weights_biases:
    print(key)
    int8_quant[key] = linear_quantize_feature(weights_biases[key], 3)[0]

fc1.weight
fc1.bias


/var/folders/j2/l2thzykn6k9489f9g899_cvc0000gn/T/ipykernel_37183/2617538202.py:90: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("cpu/NN/mnist_100_10

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.quantization
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

use_cuda = torch.cuda.is_available()


if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 64}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)


transform=transforms.Compose([
        transforms.Resize((10, 10)),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        transforms.Lambda(lambda x: torch.where(x > 0.7, torch.tensor(1.0), torch.tensor(0.0))),
        transforms.Lambda(lambda x: x.view(-1))
        ])

def filter_0_and_1(dataset):
    indices = [i for i, target in enumerate(dataset.targets) if target == 0 or target == 1]
    dataset.targets = dataset.targets[indices]
    dataset.data = dataset.data[indices]
    return dataset

test_dataset = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
# test_dataset = filter_0_and_1(test_dataset)

dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
# dataset2 = filter_0_and_1(dataset2)

# Create the DataLoader
train_loader = torch.utils.data.DataLoader(test_dataset)
test_loader = torch.utils.data.DataLoader(dataset2)

In [4]:
weights = int8_quant["fc1.weight"]
m = test_dataset[0][0].numpy().astype(int)
m

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [5]:
k = 500
N = 100

q = 2**16
p = 2**10

A = [np.random.randint(0, q, k) for _ in range(N)]
s = np.random.randint(0, 2, k)
A = np.array(A)

def polynomial_mult(s0, s1, size=N, base=q):
    result = [0] * (size)

    # Multiply the coefficients
    for i in range(len(s0)):
        for j in range(len(s1)):
            if i + j < size:
                result[i + j] += s0[i] * s1[j]

    for i in range(len(result)):
        result[i] = result[i] % base

    return result

def lwe_enc():
    # E = [1, 0, 1, -1] # \in q
    LAMBDA = 64
    delta = q / p
    # print(delta, LAMBDA, delta/LAMBDA)
    E = np.random.randint(0, delta/LAMBDA, N)
    delta_m = np.array(m) * delta

    B = (np.array(delta_m) + E)

    # breakpoint()
    for idx in range(N):
        for j in range(k):
            B[idx] += A[idx][j] * s[j]
            B[idx] %= q

    return B

def dec_lwe(A, B, s, N=100):
    for idx in range(N):
        for j in range(k):
            B[idx] -= A[idx][j] * s[j]
            B %= q
    # can check bottom bits and add one if needed
    return np.round(B / (q / p)) % q

"""
Operations on ciphertext
"""

def add_ct(ct1, ct2, A1, A2):
    return ct1 + ct2, A1 + A2

def add_constant(ct, c):
    return ct + c * q/p

def mul_constant(ct, A, c):
    return ((c * ct) % (q)), ((c * A) % (q))

In [6]:
print(m)
b = lwe_enc()
newb, newA = mul_constant(b, A, 12)
dec_lwe(newA, newb, s)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 1 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1
 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 12., 12., 12.,
       12., 12.,  0.,  0.,  0.,  0.,  0., 12., 12.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0., 12., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       12., 12.,  0.,  0.,  0.,  0.,  0.,  0., 12., 12., 12.,  0.,  0.,
        0.,  0.,  0.,  0., 12., 12., 12.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [7]:
newA = np.dot(weights, A) % q
newB = np.dot(weights, b.reshape(100, 1)) % q

# dec_lwe(newA, newB, s, N = 10)

In [8]:
new_M = np.dot(weights, m.reshape(100, 1)).reshape(10)
new_M

array([10,  1,  6, 13,  1, 14,  5,  8, 13,  5])

In [9]:
import pickle

N=100

pickle_file = "/Users/ruth/6.2050/fpga-project/nn_Asmb.pkl"

if(True):
    data = {'m': m, 's': s, 'A': nn_A, 'b': b}

    with open(pickle_file, 'wb') as f:
        pickle.dump(data, f)
    print(f"Data saved to {pickle_file}.")
else:
    with open('/Users/ruth/6.2050/fpga-project/Asm.pkl', 'rb') as f:
        loaded_data = pickle.load(f)

    A = loaded_data["A"]
    s = loaded_data["s"]
    m = loaded_data["m"]

def make_num(list, bits):
    number = 0
    #for i, val in enumerate(reversed(list)):
        #number += (val << i*bits)
    for i, val in enumerate(list):
        number += (val << i*bits)
    return number

with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/A.mem', 'w') as f:
    for x in range(N):
        for y in range(0, k, 2):
            f.write(f'{make_num(A[x][y:y+2], 16):X}\n')
print('Output image saved at A.mem')

with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/s.mem', 'w') as f:
    for y in range(0, k, 2):
        f.write(f'{make_num(s[y:y+2], 1):X}\n')
print('Output image saved at s.mem')

with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/pt.mem', 'w') as f:
    for y in range(0, N):
        f.write(f'{make_num([m[y]], 1):X}\n')
print('Output image saved at pt.mem')

b = b.astype(int)
with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/b.mem', 'w') as f:
    for x in range(N):
        f.write(f'{b[x]:X}\n')
print('Output image saved at b.mem')

NameError: name 'nn_A' is not defined

In [ ]:
pickle_file = "/Users/ruth/6.2050/fpga-project/dec_Asmb.pkl"

N = 10

if(True):
    data = {'m': new_M, 's': s, 'A': newA, "b": newB}

    with open(pickle_file, 'wb') as f:
        pickle.dump(data, f)
    print(f"Data saved to {pickle_file}.")
else:
    with open('/Users/ruth/6.2050/fpga-project/Asm.pkl', 'rb') as f:
        loaded_data = pickle.load(f)

    A = loaded_data["A"]
    s = loaded_data["s"]
    m = loaded_data["m"]

def make_num(list, bits):
    number = 0
    for i, val in enumerate(list):
        number += (val << i*bits)
    return number

with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/A.mem', 'w') as f:
    for x in range(N):
        for y in range(0, k, 2):
            f.write(f'{make_num(newA[x][y:y+2], 16):X}\n')
print('Output image saved at A.mem')

with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/s.mem', 'w') as f:
    for y in range(0, k, 2):
        f.write(f'{make_num(s[y:y+2], 1):X}\n')
print('Output image saved at s.mem')

with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/pt.mem', 'w') as f:
    for y in range(0, N):
        f.write(f'{make_num([new_M[y]], 1):X}\n')
print('Output image saved at pt.mem')

newB = newB.reshape(10).astype(int)
with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/b.mem', 'w') as f:
    for x in range(N):
        f.write(f'{newB[x]:X}\n')
print('Output image saved at b.mem')

print(newB)

Data saved to /Users/ruth/6.2050/fpga-project/dec_Asmb.pkl.
Output image saved at A.mem
Output image saved at s.mem
Output image saved at pt.mem
63332
59839
37459
54843
57837
57044
25771
35349
36422
54328
Output image saved at b.mem
[63332 59839 37459 54843 57837 57044 25771 35349 36422 54328]


In [31]:
with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/nn.mem', 'w') as f:
    
    for y in range(100):
        for x in range(10):
            f.write(f'{weights[x][y]%8:X}\n')

print('Output image saved at nn.mem')

biases = int8_quant["fc1.bias"]
with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/biases.mem', 'w') as f:
    
    for x in range(10):
        f.write(f'{biases[x]%8:X}\n')

print('Output image saved at biases.mem')

Output image saved at nn.mem
Output image saved at biases.mem


In [22]:
import pickle

nn_A = np.concatenate((A, b.reshape(100, 1)), axis=1)
nn_A = np.concatenate((nn_A, np.zeros((100, 1))), axis=1)
nn_A = nn_A.astype(int)
print(nn_A.shape)

pickle_file = "/Users/ruth/6.2050/fpga-project/nn_Asmbnn.pkl"

if(True):
    data = {'m': m, 's': s, 'A': nn_A, 'b': b, 'nn': weights}

    with open(pickle_file, 'wb') as f:
        pickle.dump(data, f)
    print(f"Data saved to {pickle_file}.")
else:
    with open('/Users/ruth/6.2050/fpga-project/Asm.pkl', 'rb') as f:
        loaded_data = pickle.load(f)

    A = loaded_data["A"]
    s = loaded_data["s"]
    m = loaded_data["m"]

def make_num(list, bits):
    number = 0
    #for i, val in enumerate(reversed(list)):
        #number += (val << i*bits)
    for i, val in enumerate(list):
        number += (val << i*bits)
    return number

with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/A.mem', 'w') as f:
    for x in range(N):
        for y in range(0, k+2, 2):
            f.write(f'{make_num(nn_A[x][y:y+2], 16):X}\n')
print('Output image saved at A.mem')

with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/s.mem', 'w') as f:
    for y in range(0, k, 2):
        f.write(f'{make_num(s[y:y+2], 1):X}\n')
print('Output image saved at s.mem')

with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/pt.mem', 'w') as f:
    for y in range(0, N):
        f.write(f'{make_num([m[y]], 1):X}\n')
print('Output image saved at pt.mem')

b = b.astype(int)
with open(f'/Users/ruth/6.2050/fpga-project/lab03/data/b.mem', 'w') as f:
    for x in range(2510):
        f.write(f'{0:X}\n')
print('Output image saved at b.mem')

print(b)
print(nn_A)

(100, 502)
Data saved to /Users/ruth/6.2050/fpga-project/nn_Asmbnn.pkl.
Output image saved at A.mem
Output image saved at s.mem
Output image saved at pt.mem
Output image saved at b.mem
[13024 21428 58183 65357 28990 29621 43803 24930 33551 60489  1902 26220
 24318 35089 14492  9963  7116 60617 36329 28418 42640 56014 22735 10489
 29499 32334 31074 35953 42967 20426 34898 57658 13964 62367 64827 38811
 27626  9635 18395 62679 44146 21821 31397 10575 38856 63216 42725  9739
  2839 21104 53856  3587 30637 55027 63228 54268 64856    84 47189 10271
 12765 28951  6962 53148 60440 56089 19927 55128 17652 12548 59903 15465
  5023 58893 22431 22444 31497 42816 45694 12459  1783 48371 36094 29660
 34342 40726 32490 59336 64764 45888 50451 42741 23221 34954 43803 29681
 35717 30921 11178 29562]
[[ 1646 44401 31295 ... 57867 13024     0]
 [45642 51371  4208 ...  4528 21428     0]
 [55371  4285 12767 ... 31955 58183     0]
 ...
 [35872 33764 41191 ... 46707 30921     0]
 [45248  5029 36112 ... 5710

In [24]:
print(newA)
print(newB)

print(nn_A.shape)

[[52774 37069 17388 ... 21463 24426 45582]
 [15765  3787 62161 ... 14539 63898 50982]
 [44216  8382 12056 ...   696 27930 11808]
 ...
 [43045 44433  2966 ...   666 41177 53506]
 [50853 41238  4730 ... 27675 48747 61899]
 [  525 32797 36581 ... 54958 13186 40604]]
[[36237.]
 [64280.]
 [24134.]
 [12073.]
 [37135.]
 [27133.]
 [45598.]
 [ 6287.]
 [56726.]
 [ 8956.]]
(100, 502)


In [29]:
biases = int8_quant["fc1.bias"]
bias_B = newB+biases.reshape(10, 1).astype(int)*2**6
bias_B

array([[36045.],
       [64408.],
       [23942.],
       [11945.],
       [37071.],
       [27069.],
       [45470.],
       [ 6287.],
       [56470.],
       [ 8764.]])

In [30]:
dec_lwe(newA, bias_B, s, N=10)

array([[ 7.],
       [ 3.],
       [ 3.],
       [11.],
       [ 0.],
       [13.],
       [ 3.],
       [ 8.],
       [ 9.],
       [ 2.]])

In [28]:
np.dot(weights, m) + biases

array([ 7,  3,  3, 11,  0, 13,  3,  8,  9,  2])

In [54]:
dec2(A[:, v].reshape(500, 1), [enc()[v]], s[:, v].reshape(500, 1), 1)


array([423.])

In [40]:
A.T[v, :].T.shape

(500,)

In [49]:
A.shape

(500, 100)

In [ ]:
def sample_extraction(h = 0):

    N = 2
    k = 2

    res_ct = [[1, 4, 6], [3, 5, 7]]
    b = 0
    h = 1

    res_ct.append([0]*k*N)

    for i in range(k):
        for j in range(h+1): 
            res_ct[N][i+j] = res_ct[i][h-j]
    for i in range(k):
        for j in range(h+1, N): 
            res_ct[N][i+j] = res_ct[i][h-j + N]

    print(res_ct)
    b = b

sample_extraction()

[[1, 4, 6], [3, 5, 7], [4, 5, 3, 0]]
